In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('.secret/yelp_api.json') as f:
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# declare variables
LOCATION = 'SAN ANTONIO, TX'
TERM = 'Pizza'

In [4]:
# save json file
JSON_FILE = "Data/results_in_progress_SAN_ANTONIO_pizza.json"
JSON_FILE

'Data/results_in_progress_SAN_ANTONIO_pizza.json'

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_SAN_ANTONIO_pizza.json not found. Saving empty list to file.


In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [8]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [11]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [12]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

65

In [13]:
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [14]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [15]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_SAN_ANTONIO_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


65

In [16]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/65 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [17]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,mE8oeT-264q0MRKhVo7j8A,dough-pizzeria-napoletana-san-antonio,Dough Pizzeria Napoletana,https://s3-media2.fl.yelpcdn.com/bphoto/nMZExv...,False,https://www.yelp.com/biz/dough-pizzeria-napole...,1809,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 29.5195143, 'longitude': -98.5069...","[delivery, pickup]",$$,"{'address1': '6989 Blanco Rd', 'address2': '',...",+12109796565,(210) 979-6565,2850.666555
1,BgoLUf0OJSdkaTDeVwz2MA,wild-barley-kitchen-and-brewery-san-antonio,Wild Barley Kitchen & Brewery,https://s3-media1.fl.yelpcdn.com/bphoto/eZGu45...,False,https://www.yelp.com/biz/wild-barley-kitchen-a...,106,"[{'alias': 'brewpubs', 'title': 'Brewpubs'}, {...",4.5,"{'latitude': 29.51537923739746, 'longitude': -...",[],$$,"{'address1': '8403 Broadway', 'address2': None...",+12104559982,(210) 455-9982,5249.162072
2,OeLJy69rYAb0Rq0feRl2yw,pizza-break-san-antonio,Pizza Break,https://s3-media2.fl.yelpcdn.com/bphoto/HMabaZ...,False,https://www.yelp.com/biz/pizza-break-san-anton...,17,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,"{'latitude': 29.5516345, 'longitude': -98.5117...","[delivery, pickup]",NaN,"{'address1': '12072 Blanco Rd', 'address2': 'S...",+12107018086,(210) 701-8086,6365.084721
3,AX2NYuKt1fx0Q97Muf4TUw,mattengas-pizzeria-callaghan-san-antonio-2,Mattenga's Pizzeria - Callaghan,https://s3-media1.fl.yelpcdn.com/bphoto/Pw6jKa...,False,https://www.yelp.com/biz/mattengas-pizzeria-ca...,36,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 29.50992, 'longitude': -98.55533}",[],NaN,"{'address1': '7863 Callaghan Rd', 'address2': ...",+12104740065,(210) 474-0065,4253.353417
4,Ht62-ZGuv8gzRJ_pO-kV_w,via-313-pizza-gateway-san-antonio,Via 313 Pizza - Gateway,https://s3-media3.fl.yelpcdn.com/bphoto/qkLOUa...,False,https://www.yelp.com/biz/via-313-pizza-gateway...,216,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 29.520115, 'longitude': -98.5745}",[],$$,"{'address1': '8435 Wurzbach Rd', 'address2': '...",+12109851313,(210) 985-1313,6467.581173


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,j4Fbase3B_pSSGfvsUcRtA,panera-bread-san-antonio-14,Panera Bread,https://s3-media2.fl.yelpcdn.com/bphoto/3WVE0S...,False,https://www.yelp.com/biz/panera-bread-san-anto...,38,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",2.0,"{'latitude': 29.5567173376684, 'longitude': -9...","[pickup, delivery]",NaN,"{'address1': '9106 N Loop 1604 W', 'address2':...",+12107755865,(210) 775-5865,15863.575085
996,NGNgluVc-_BjoJ3cx8z8og,7-eleven-san-antonio-14,7-Eleven,https://s3-media3.fl.yelpcdn.com/bphoto/8BSgN3...,False,https://www.yelp.com/biz/7-eleven-san-antonio-...,3,"[{'alias': 'servicestations', 'title': 'Gas St...",4.5,"{'latitude': 29.5528233, 'longitude': -98.3562...","[pickup, delivery]",$,"{'address1': '12351 N Interstate 35', 'address...",+12105902532,(210) 590-2532,16624.281932
997,42rPSfx8X7aIZvhLQbOabQ,olive-garden-italian-restaurant-live-oak-3,Olive Garden Italian Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/RndQYZ...,False,https://www.yelp.com/biz/olive-garden-italian-...,78,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",3.5,"{'latitude': 29.5635916, 'longitude': -98.3408...",[],$$,"{'address1': '2150 Ikea-rbfcu Pkwy', 'address2...",+12106460887,(210) 646-0887,18471.175298
998,A6uDJmMR9jXcC-CPSdV31w,7-eleven-helotes,7-Eleven,https://s3-media3.fl.yelpcdn.com/bphoto/k8pziF...,False,https://www.yelp.com/biz/7-eleven-helotes?adju...,4,"[{'alias': 'convenience', 'title': 'Convenienc...",1.0,"{'latitude': 29.5552970758725, 'longitude': -9...","[pickup, delivery]",$,"{'address1': '11808 Bandera Rd', 'address2': '...",+12103729014,(210) 372-9014,16305.736053
999,7rqQuxhZcHfusenxQ2WisA,jaimes-place-san-antonio,Jaime's Place,https://s3-media2.fl.yelpcdn.com/bphoto/uItef5...,False,https://www.yelp.com/biz/jaimes-place-san-anto...,21,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 29.42645, 'longitude': -98.51168}",[],$$,"{'address1': '1514 West Commerce St', 'address...",+12105649083,(210) 564-9083,7578.028706


In [18]:
# check for duplicates
final_df.duplicated(subset='id').sum()

0

In [19]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)